## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np
import numba

In [2]:
%load_ext line_profiler

In [3]:
A = np.random.randint(0,1000, size=(1000000,))
A

array([605, 112, 713, ..., 383, 934, 402])

In [4]:
def f1(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai +100
        acc += bi
        cnt += 1
    return acc/ cnt

In [5]:
%timeit f1(A)

147 ms ± 4.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
def f2(A):
    acc = 0
    for ai in A:
        bi = ai +100
        acc += bi
    return acc / len(A)
        

In [7]:
%timeit f2(A)

128 ms ± 4.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%lprun -f f2 f2(A)

In [9]:
%lprun -f f1 f1(A)

In [10]:
def f3(A):
    return A.mean()+100

In [11]:
%timeit f3(A)

332 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%lprun -f f3 f3(A)

In [13]:
@numba.njit
def f5(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai +100
        acc += bi
        cnt += 1
    return acc/cnt

In [14]:
%timeit f5(A)

333 µs ± 701 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [15]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)


def g_optimize(df):
    return df[df["key"].str.contains("a|b|c|d|e")]

def g_optimize_v2(df):
    return df[df["key"].isin(('a', 'b', 'c', 'd', 'e', 'f', 'g'))]
    

In [16]:
%timeit g(df)

374 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit g_optimize(df)

396 ms ± 6.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit g_optimize_v2(df)

60.7 ms ± 922 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

In [19]:
# !pip install line_profiler

In [20]:
import pandas as pd

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [21]:
recipes = pd.read_csv("./data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')
recipes

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...
267661,zurie s holey rustic olive and cheddar bread,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
386977,zwetschgenkuchen bavarian plum cake,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
103312,zwiebelkuchen southwest german onion cake,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN


In [22]:
recipes.dtypes

name                      object
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

In [23]:
reviews = pd.read_csv("./data/reviews_sample.csv", sep=",",parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')
reviews

,user_id,recipe_id,date,rating,review
id,,,,,
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."


In [24]:
reviews.dtypes

user_id               int64
recipe_id             int64
date         datetime64[ns]
rating                int64
review               object
dtype: object

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

In [25]:
def f1_test():
    counter = 0
    values = 0
    for index, row in reviews.iterrows():
        if row["date"].year == 2010:
            values += row["rating"]
            counter += 1

    return values/counter

result1 = f1_test()
result1

4.4544402182900615

In [26]:
%timeit f1_test()

1.96 s ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

In [27]:
def f2_test():
    counter = 0
    values = 0
    selected_year_df = reviews[reviews['date'].dt.year == 2010]
    for index, row in selected_year_df.iterrows():
        values += row["rating"]
        counter += 1

    return values/counter
    
result2 = f2_test()
result2

4.4544402182900615

In [28]:
%timeit f2_test()

190 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


В. С использованием метода `DataFrame.mean`

In [29]:
def f3_test():
    selected_year_df = reviews['date'].dt.year == 2010
    return reviews.loc[selected_year_df, 'rating'].mean()

result3 = f3_test()
result3

4.4544402182900615

In [30]:
%timeit f3_test()

5.24 ms ± 70.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
np.allclose(result1, result2)

True

In [32]:
np.allclose(result1, result3)

True

In [33]:
np.allclose(result2, result3)

True

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [34]:
%lprun -f f1_test f1_test()

Timer unit: 1e-06 s

Total time: 7.91871 s

Could not find file /var/folders/wp/b9w68jpd68xfy1wmgy4_08xh0000gn/T/ipykernel_13001/2028688920.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1          3.0      3.0      0.0  
     3         1          1.0      1.0      0.0  
     4    126697    7141930.0     56.4     90.2  
     5    126696     712141.0      5.6      9.0  
     6     12094      61773.0      5.1      0.8  
     7     12094       2857.0      0.2      0.0  
     8                                           
     9         1          1.0      1.0      0.0

In [35]:
%lprun -f f2_test f2_test()

Timer unit: 1e-06 s

Total time: 0.771786 s

Could not find file /var/folders/wp/b9w68jpd68xfy1wmgy4_08xh0000gn/T/ipykernel_13001/3568662651.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           
     3         1          2.0      2.0      0.0  
     4         1          1.0      1.0      0.0  
     5         1      14458.0  14458.0      1.9  
     6     12095     685600.0     56.7     88.8  
     7     12094      68965.0      5.7      8.9  
     8     12094       2759.0      0.2      0.4  
     9                                           
    10         1          1.0      1.0      0.0

In [36]:
%lprun -f f3_test f3_test()

Timer unit: 1e-06 s

Total time: 0.015072 s

Could not find file /var/folders/wp/b9w68jpd68xfy1wmgy4_08xh0000gn/T/ipykernel_13001/3384476274.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1      13307.0  13307.0     88.3  
     3         1       1765.0   1765.0     11.7

### Пример еще более быстрой функции

In [37]:
import numpy as np

In [38]:
def f4_test():
    
    data = np.matrix((reviews["rating"], reviews["date"].dt.year), dtype=int)
    #Маска
    mask = data[1,] == 2010
    #Применяем элементы для маски
    faster_values = np.where(mask, data, 0)
    #Получаем результат
    return faster_values[0,].sum() / mask.sum()

In [39]:
f4_test()

4.4544402182900615

In [40]:
%timeit f4_test()

5.58 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
%lprun -f f4_test f4_test()

In [42]:
reviews_rating = np.array(reviews["rating"])
reviews_other = np.array(reviews["date"].dt.year)

In [43]:

def f5_test():
    
    data = np.matrix((reviews_rating, reviews_other), dtype=int)
    #Маска
    mask = data[1,] == 2010
    #Применяем элементы для маски
    faster_values = np.where(mask, data, 0)
    #Получаем результат
    return faster_values[0,].sum() / mask.sum()

In [44]:
f5_test()

4.4544402182900615

In [45]:
%timeit f5_test()

460 µs ± 60.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [46]:
def get_word_reviews_count(df):
    
    word_reviews = {}
    
    #Удаляем нулевые и проходим идерацией
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        #Делаем разделение по словами 
        words = review.split(' ')
        #По каждому слову начинаем цикл
        for word in words:
            #Если слово не в словаре, то заносим его
            if word not in word_reviews:
                word_reviews[word] = []
            
            #Добавляем рецепт по этому слову
            word_reviews[word].append(recipe_id)
    
    
    word_reviews_count = {}
    #Удаляем нулевые и проходим идерацией еще разок
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        # Проходим по каждому слову
        for word in words:
            #Заносим кол-во отзывов через len 
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [47]:
get_word_reviews_count(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

1) Второй цикл for _, row in df.dropna(subset=['review']).iterrows(): не нужен, можно работать через items word_reviews_count[word] = len(word_reviews[word])<br>
2) Использование iterrows в pd - крайний вариант, медленный сам по себе<br>
3) Не понятно то, зачем используем буферный word_reviews, а только потом word_reviews_count, может сразу его решим<br>

In [48]:
%timeit get_word_reviews_count(reviews)

6.58 s ± 278 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-06 s

Total time: 26.4647 s

Could not find file /var/folders/wp/b9w68jpd68xfy1wmgy4_08xh0000gn/T/ipykernel_13001/2826575548.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1          3.0      3.0      0.0  
     3    126680    7245483.0     57.2     27.4  
     4    126679    1279192.0     10.1      4.8  
     5    126679     264358.0      2.1      1.0  
     6   6918689    1414207.0      0.2      5.3  
     7   6792010    1992209.0      0.3      7.5  
     8    174944      56594.0      0.3      0.2  
     9   6792010    2095395.0      0.3      7.9  
    10                                           
    11         1          0.0      0.0      0.0  
    12    126680    7096978.0     56.0     26.8  
    13    126679     722193.0      5.7      2.7  
    14    126679     263010.0      2.1      1.0  
    15   6918689    1438921.0      0.2      5.4  
    16   6792010    2596202.0      0.4      9.8  
    17         1          1.0      1.0      0.0

In [50]:
def get_word_reviews_count_optimized(df):
    
    word_reviews = {}
    
    #Удаляем нулевые и проходим идерацией
    for _, row in df.dropna(subset=['review']).iterrows():

        #По каждому слову начинаем цикл
        for word in row['review'].split(' '):
            #Если слово не в словаре, то заносим его
            if word not in word_reviews:
                word_reviews[word] = 0
            #Добавляем рецепт по этому слову
            word_reviews[word] += 1
    
    return word_reviews

In [51]:
get_word_reviews_count_optimized(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [52]:
%timeit get_word_reviews_count_optimized(reviews)

3.05 s ± 35.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%lprun -f get_word_reviews_count_optimized get_word_reviews_count_optimized(reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [54]:
buffer = reviews[['recipe_id', 'rating']].dropna()
mask = buffer['rating'] != 0
buffer = buffer[mask].groupby(buffer['recipe_id'])['rating']

In [55]:
#Удаление из выборки отзывов с нулевым рейтингом

In [56]:
def executor(series_data, function):
    """Метод для вызова функции c аргументами"""
    return function(series_data.to_numpy(), series_data.mean())

In [57]:
def MAPE_FIRST(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

#Конвертация dataframe в series
result1 = buffer.agg(executor, MAPE_FIRST)

In [58]:
%timeit buffer.agg(executor, MAPE_FIRST)

714 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
@numba.jit(nopython=True)
def MAPE_SECOND(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

result2 = buffer.agg(executor, MAPE_SECOND)

In [60]:
%timeit buffer.agg(executor, MAPE_SECOND)

569 ms ± 3.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
np.allclose(result1, result2)

True